In [ ]:
import pandas as pd
import sqlite3

## Создаю соединение с файлом БД

In [ ]:
con = sqlite3.connect('.\content\indicators_db.sqlite3')

Создаю функциональную обертку для удобства запросов в БД

In [ ]:
def read_sql(sql_query, connection=con):
    return pd.read_sql(sql= sql_query, con=connection)

В переменную буду передавать запросы

In [ ]:
sql_q = 'select * from main.org_indicators_2020_2022'

In [ ]:
df= read_sql(sql_q)
df.head()

# Загрузка дополнительных таблиц

In [ ]:
df_main = pd.read_csv('./content/main.csv')
df_net_property = pd.read_csv('./content/net_property.csv')
df_seats_property = pd.read_csv('./content/seats_property.csv')
df_type_property = pd.read_csv('./content/type_property.csv')
# Актуализирую таблицу с посадочными местами
df_seats_property = df_seats_property.sort_values(by=['ID', 'ver_ID'], ascending=[True, False]) \
    .drop_duplicates('ID', keep='first')

#### <i>Задание 1. Для каждой организации из main.csv подтяните выручку и число сотрудников за 2020, 2021 и 2022. Для каждой организации из main.csv подтяните выручку и число сотрудников за 2020, 2021 и 2022. Посчитайте дельту по выручке и сотрудникам между 2022 и 2020. Отсортируйте организации по убыванию выручки. Для организаций с одинаковой выручкой выполните сортировку по возрастанию ID. Если у организации нет данных по выручке или числу сотрудников за 2020 или 2022, то исключите их. Названия колонок сделайте на русском языке.</i>

##### В SQL запросе сразу провожу агрегацию VALUE группировкой по колонкам  ID YEAR INDICATOR_NAME. Фильтрую INDICATOR_NAME  по Выручке и числу сотрудников

In [ ]:
sql_q = '''select ID, YEAR, INDICATOR_NAME, SUM(VALUE) as VALUE
from main.org_indicators_2020_2022
where INDICATOR_NAME in ('Выручка', 'Число сотрудников')
group by ID, YEAR, INDICATOR_NAME;'''
# sql_q = '''select *
# from main.org_indicators_2020_2022
# where INDICATOR_NAME in ('Выручка', 'Число сотрудников');'''
df = read_sql(sql_q)
df.head(20)

In [41]:
t1 = df.pivot_table(index='ID', columns=['INDICATOR_NAME', 'YEAR'], values='VALUE')
t1.head()

INDICATOR_NAME     Выручка                         Число сотрудников        \
YEAR                  2020        2021        2022              2020  2021   
ID                                                                           
19630           31222228.0  21721921.0         NaN               NaN  15.0   
19643           35670701.0  39125612.0         NaN              17.0   NaN   
19647           34926614.0  23176004.0  27925131.0              16.0  15.0   
19657           23410468.0  28065717.0  36074059.0               NaN   8.0   
19677                  NaN  30803884.0  25303592.0              17.0   9.0   

INDICATOR_NAME        
YEAR            2022  
ID                    
19630            9.0  
19643            NaN  
19647            NaN  
19657           12.0  
19677            NaN

##### Преобразование имен колонок. Убираю мультииндекс

In [62]:
t1.columns = [x+' '+str(y) for x, y in t1.columns]
t1.head()

,Выручка 2020,Выручка 2021,Выручка 2022,Число сотрудников 2020,Число сотрудников 2021,Число сотрудников 2022
ID,,,,,,
19630,31222228.0,21721921.0,NaN,NaN,15.0,9.0
19643,35670701.0,39125612.0,NaN,17.0,NaN,NaN
19647,34926614.0,23176004.0,27925131.0,16.0,15.0,NaN
19657,23410468.0,28065717.0,36074059.0,NaN,8.0,12.0
19677,NaN,30803884.0,25303592.0,17.0,9.0,NaN


In [63]:
t1.reset_index(inplace=True)

Объединяю 2 датафрейма (Main и t1)

In [66]:
df1 = df_main.merge(t1, on='ID')
df1.head()

,ID,Name,Address,Выручка 2020,Выручка 2021,Выручка 2022,Число сотрудников 2020,Число сотрудников 2021,Число сотрудников 2022
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15",29337447.0,24299167.0,31395846.0,NaN,NaN,15.0
1,24828,Кафе Халяль,"город Москва, 8-я улица Текстильщиков, дом 5, ...",36633885.0,31262894.0,33854306.0,NaN,10.0,NaN
2,358126,Scarface,"Российская Федерация, город Москва, внутригоро...",NaN,20278817.0,26250473.0,19.0,16.0,6.0
3,373052,Кулинариум,"Российская Федерация, город Москва, внутригоро...",36385240.0,26416139.0,39003318.0,16.0,6.0,NaN
4,303239,One price coffee,"Российская Федерация, город Москва, внутригоро...",25968809.0,30183522.0,NaN,13.0,17.0,18.0


Считаю дельты по годам. Вычитаю из года более позднего (2022), год более ранний (2020)

In [67]:
df1['Дельта по выручке'] = df1['Выручка 2022'] - df1['Выручка 2020']
df1['Дельта по сотрудникам'] = df1['Число сотрудников 2022'] - df1['Число сотрудников 2020']

Переименование столбцов на русский язык.

In [68]:
df1 = df1.rename(columns={'ID': 'Идентификатор', 'Name': 'Наименование', 'Address': 'Адрес'})
df1 = df1[['Идентификатор', 'Наименование', 'Адрес', 'Выручка 2020',
       'Выручка 2021', 'Выручка 2022', 'Дельта по выручке', 'Число сотрудников 2020',
       'Число сотрудников 2021', 'Число сотрудников 2022',
       'Дельта по сотрудникам']]
df1.head()

,Идентификатор,Наименование,Адрес,Выручка 2020,Выручка 2021,Выручка 2022,Дельта по выручке,Число сотрудников 2020,Число сотрудников 2021,Число сотрудников 2022,Дельта по сотрудникам
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15",29337447.0,24299167.0,31395846.0,2058399.0,NaN,NaN,15.0,NaN
1,24828,Кафе Халяль,"город Москва, 8-я улица Текстильщиков, дом 5, ...",36633885.0,31262894.0,33854306.0,-2779579.0,NaN,10.0,NaN,NaN
2,358126,Scarface,"Российская Федерация, город Москва, внутригоро...",NaN,20278817.0,26250473.0,NaN,19.0,16.0,6.0,-13.0
3,373052,Кулинариум,"Российская Федерация, город Москва, внутригоро...",36385240.0,26416139.0,39003318.0,2618078.0,16.0,6.0,NaN,NaN
4,303239,One price coffee,"Российская Федерация, город Москва, внутригоро...",25968809.0,30183522.0,NaN,NaN,13.0,17.0,18.0,5.0


Сортировка по заданию, удаление NaN по заданию.

In [69]:
df1 = df1.sort_values(by=['Дельта по выручке', 'Идентификатор'], ascending=[False, True])
df_result = df1.dropna(subset=['Выручка 2020', 'Выручка 2022', 'Число сотрудников 2020', 'Число сотрудников 2022'])
df_result.head()

,Идентификатор,Наименование,Адрес,Выручка 2020,Выручка 2021,Выручка 2022,Дельта по выручке,Число сотрудников 2020,Число сотрудников 2021,Число сотрудников 2022,Дельта по сотрудникам
9205,342920,СИДРОВ ПРОЛИВ (пивной бар),"город Москва, проспект Вернадского, дом 86В",20302399.0,NaN,39959846.0,19657447.0,6.0,11.0,14.0,8.0
9596,374427,Море лосося,"город Москва, улица Адмирала Макарова, дом 6, ...",20000135.0,NaN,39484937.0,19484802.0,7.0,NaN,19.0,12.0
5890,360584,"Выпечка, шаурма","Российская Федерация, город Москва, внутригоро...",20114682.0,23403344.0,39418028.0,19303346.0,13.0,NaN,11.0,-2.0
8745,344731,Кофе с собой,"Российская Федерация, город Москва, внутригоро...",21020009.0,39682516.0,39851594.0,18831585.0,8.0,NaN,10.0,2.0
4904,373914,Кафе,"город Москва, улица Айвазовского, владение 7, ...",20663231.0,31079091.0,38834892.0,18171661.0,7.0,16.0,14.0,7.0


Сохранение результатов в CSV файл

In [70]:
df_result.to_csv('ex_6_1_1.csv', index=False)

In [73]:
df_old = pd.read_csv('ex_6_1.csv')

In [85]:
print((df_result.values == df_old.values))

[[ True  True  True ...  True  True  True]
 [ True  True  True ... False  True  True]
 [ True  True  True ... False  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ... False  True  True]
 [ True  True  True ...  True  True  True]]
